<a href="https://colab.research.google.com/github/Semalign/Semalign/blob/main/MachineLearningModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# here we begin by defining our problem
it is diabetes progression based on
various medical features


In [1]:
# Goal : predict the progression of diabetes(regression problem)
# data : synthetic using random module from numpy
# sucess metrics : MSE


In [10]:
# step 1 : create synthetic data using numpy and pandas
import numpy as np
import pandas as pd

num_samples=100
np.random.seed(10)
age=np.random.randint(20,80,num_samples)
bmi=np.random.uniform(18,40,num_samples)
blood_pressure=np.random.uniform(80,180,num_samples)
m1=np.random.uniform(0.2,6.2,num_samples)
m2=np.random.uniform(0.2,6.2,num_samples)
m3=np.random.uniform(0.2,6.2,num_samples)

progression=(50+age*0.3+bmi*2+blood_pressure*0.1+m1*2+m2*3+m3*1.5)+ np.random.randint(-10,10,num_samples)

diabetes_data=pd.DataFrame({
    'age':age,
    'bmi':bmi,
    'blood_pressure':blood_pressure,
    'm1':m1,
    'm2':m2,
    'm3':m3,
    'progression':progression,
})
diabetes_data.head()
diabetes_data.to_csv('synthetic_diabetes_data',index=False)

In [12]:
# the next step is preparing the data for training
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

data=pd.read_csv('synthetic_diabetes_data.csv')
print(data.head())


   age        bmi  blood_pressure        m1        m2        m3  progression
0   29  36.064324      130.697863  5.016656  1.507451  0.829744   161.698715
1   56  21.325344      161.903586  3.965371  5.619030  3.263788   150.324563
2   35  26.450518       89.010673  5.127299  0.258620  4.072932   136.441958
3   20  38.773736      160.006875  1.929519  1.554876  1.118401   157.749427
4   69  39.727760      136.512635  1.848771  0.992331  0.210558   178.797155


In [14]:
X=data.drop('progression', axis=1)
y=data['progression']

numerical_features=['age','bmi','blood_pressure','m1','m2','m3']
numeric_transformer=Pipeline(steps=[('scaler',StandardScaler())])

preprocessor=ColumnTransformer(transformers=[('num',numeric_transformer,numerical_features)])


In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.fit_transform(X_test)


In [24]:
# now the next step is training the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf_model =RandomForestRegressor(n_estimators=100,random_state=10)
rf_model.fit(X_train,y_train)

y_pred=rf_model.predict(X_test)

mse=mean_squared_error(y_test,y_pred)

print(f'mse:{mse}')

mse:97.3167416843925


In [ ]:
# now after the evaluation the next step is to evaluate the model
# we can use grid search to obtain the best hyperparametrs

from sklearn.model_selection import GridSearchCV
param_grid={
    'n_estimators':[50,100,200],
    'max_depth':[None,10,20,30],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4],
}
grid_search=GridSearchCV(estimator=RandomForestRegressor(random_state=10),param_grid=param_grid,cv=5,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

best_params=grid_search.best_params_
print(f'best parameters:{best_params}')
best_rf_model=grid_search.best_estimator_
best_rf_y_pred=best_rf_model.predict(X_test)
best_rf_mse=mean_squared_error(y_test,best_rf_y_pred)
print(f'mse:{mse}')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
